In [1]:
%matplotlib inline
import pycistarget
pycistarget.__version__

import os
import glob
import pandas as pd
import matplotlib.pyplot as plt

# Load cistarget functions
from pycistarget.motif_enrichment_cistarget import *

import pyranges as pr
from pycistarget.utils import *

/home/jovyan/my-conda-envs/ni_env2/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-12-07 15:53:12,915	INFO util.py:159 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
## Defining paths to input/output and cistarget db
input_dir = '/lustre/scratch126/cellgen/team205/sk29/matthias_fb/bulk_ATACseq_pipeline/tables/'
cistarget_db = '/lustre/scratch126/cellgen/team205/is10/fibroblasts/bulk_cytokines/cistarget/cistarget_db.regions_vs_motifs.rankings.feather'
motif_anno = '/nfs/team205/is10/resources/aerts_motifs/v10nr_clust_public/snapshots/motifs-v10-nr.hgnc-m0.00001-o0.0.tbl'
out_dir = 'enrichment_padj0.05/'
os.makedirs(out_dir, exist_ok=True)

In [3]:
## Cutoffs for padj and logFC for DA peaks
padj_cutoff = 0.05
logFC_cutoff = 0.5

In [4]:
files = os.listdir(input_dir)
condition_paths = {}
for f in files:
    if ('differential' in f) and ('.csv' in f):
        condition_paths[f.split('.')[0]] = (input_dir + f)

In [5]:
condition_paths

{'differential_peak_table_FDR05_annotated_matched_Cytokine': '/lustre/scratch126/cellgen/team205/sk29/matthias_fb/bulk_ATACseq_pipeline/tables/differential_peak_table_FDR05_annotated_matched_Cytokine.csv',
 'differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat': '/lustre/scratch126/cellgen/team205/sk29/matthias_fb/bulk_ATACseq_pipeline/tables/differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat.csv',
 'differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine': '/lustre/scratch126/cellgen/team205/sk29/matthias_fb/bulk_ATACseq_pipeline/tables/differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine.csv'}

In [6]:
#peaks[(peaks['FDR'] < padj_cutoff) & (peaks['logFC'] >= logFC_cutoff)]

In [7]:
#peaks

In [8]:
#cond

In [9]:
region_sets = {}

for cond in condition_paths.keys():
    peaks = pd.read_csv(condition_paths[cond])

    ## Up
    up = peaks[(peaks['FDR'] < padj_cutoff) & (peaks['logFC'] >= logFC_cutoff)]
    up_peaks = (up['seqnames'] + ":" + up['start'].astype(str) + '-' + up['end'].astype(str)).tolist()
    
    if len(up_peaks) > 0:
        region_sets[(cond + '_up')] = pr.PyRanges(region_names_to_coordinates(up_peaks))

    ## Down
    down = peaks[(peaks['FDR'] < padj_cutoff) & (peaks['logFC'] < ((-1) * logFC_cutoff))]
    down_peaks = (down['seqnames'] + ":" + down['start'].astype(str) + '-' + down['end'].astype(str)).tolist() 
    
    if (len(down_peaks) > 0):
        region_sets[(cond + '_down')] = pr.PyRanges(region_names_to_coordinates(down_peaks))

/home/jovyan/my-conda-envs/ni_env2/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/home/jovyan/my-conda-envs/ni_env2/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/home/jovyan/my-conda-envs/ni_env2/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain c

In [10]:
region_sets

{'differential_peak_table_FDR05_annotated_matched_Cytokine_up': +--------------+-----------+-----------+
 | Chromosome   | Start     | End       |
 | (category)   | (int64)   | (int64)   |
 |--------------+-----------+-----------|
 | chr1         | 81440818  | 81441170  |
 | chr1         | 209720330 | 209721193 |
 | chr1         | 81424081  | 81424543  |
 | chr1         | 81414458  | 81414870  |
 | ...          | ...       | ...       |
 | chrX         | 151912120 | 151913369 |
 | chrX         | 71053127  | 71053590  |
 | chrX         | 19000997  | 19001179  |
 | chrX         | 104232381 | 104232640 |
 | chrY         | 10942606  | 10942976  |
 +--------------+-----------+-----------+
 Unstranded PyRanges object has 6,055 rows and 3 columns from 24 chromosomes.
 For printing, the PyRanges was sorted on Chromosome.,
 'differential_peak_table_FDR05_annotated_matched_Cytokine_down': +--------------+-----------+-----------+
 | Chromosome   | Start     | End       |
 | (category)   | (int64)

In [11]:
# Run
cistarget_dict = run_cistarget(ctx_db = cistarget_db,
                               region_sets = region_sets,
                               specie = 'homo_sapiens',
                               annotation_version = 'v10nr_clust',
                               path_to_motif_annotations = motif_anno,
                               auc_threshold = 0.005,
                               nes_threshold = 3.0,
                               rank_threshold = 0.05,
                               annotation = ['Direct_annot', 'Orthology_annot'],
                               n_cpu = 1,
                               _temp_dir='tmp/')

2023-12-07 15:53:14,571 cisTarget    INFO     Reading cisTarget database


/home/jovyan/my-conda-envs/ni_env2/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/home/jovyan/my-conda-envs/ni_env2/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  return {k: v for k, v in df.groupby(grpby_key)}
/home/jovyan/my-conda-envs/ni_env2/lib/python3.10/site-packages/pyranges/methods/init.py:45: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain c

2023-12-07 15:54:17,956 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine_up which has 7601 regions
2023-12-07 15:54:27,677 cisTarget    INFO     Annotating motifs for differential_peak_table_FDR05_annotated_matched_Cytokine_up
2023-12-07 15:54:30,126 cisTarget    INFO     Getting cistromes for differential_peak_table_FDR05_annotated_matched_Cytokine_up
2023-12-07 15:54:30,286 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine_down which has 89949 regions
2023-12-07 15:54:41,535 cisTarget    INFO     No enriched motifs found for differential_peak_table_FDR05_annotated_matched_Cytokine_down
2023-12-07 15:54:41,553 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up which has 8396 regions
2023-12-07 15:54:47,114 cisTarget    INFO     Annotating motifs for differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_

In [12]:
# Save
import pickle
with open((out_dir + 'cistarget_dict_NES3.0.pkl'), 'wb') as f:
  pickle.dump(cistarget_dict, f)

In [13]:
## Also exporting results in a table format:
motif_enrichment_dict = {key: cistarget_dict[key].motif_enrichment for key in cistarget_dict.keys()}
motif_enrichment_table=pd.concat([motif_enrichment_dict[key] for key in motif_enrichment_dict.keys()], axis=0, sort=False)
motif_enrichment_table.to_csv((out_dir + "cistarget_res_df_NES3.0.txt"), sep="\t")

In [22]:
cistarget_results(cistarget_dict, name="differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down")

,Logo,Region_set,Direct_annot,Orthology_annot,NES,AUC,Rank_at_max,Motif_hits
metacluster_43.2,,differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down,"NEUROD1, ZBTB42, MSC, ZBTB18, MESP2, NEUROG2, TWIST1","PTF1A, ATOH1, NEUROD2",13.624985,0.029691,16646.0,506
homer__NAACAGCTGG_Tcf21,,differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down,TCF21,NaN,13.402956,0.029253,16740.0,468
homer__NAHCAGCTGD_Ap4,,differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down,TFAP4,NaN,12.958118,0.028377,16688.0,476
metacluster_193.3,,differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down,"TWIST1, TCF3","TWIST2, BHLHA15",12.592148,0.027655,16193.0,476
metacluster_43.1,,differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down,"NEUROD1, NEUROD2, OLIG2, TAL1, MSC, ATOH1, PTF1A, NEUROG2, MYOG, BHLHA15, TCF12","NEUROD1, OLIG2, NEUROG3, OLIG3, MYOD1, BHLHE22, NEUROD4, NEUROD6, OLIG1, ATOH1, ATOH7, BHLHE23, NEUROG2, TCF12, NEUROG1, NEUROD2",12.219801,0.026921,15978.0,445
hdpi__ZNF238,,differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down,ZBTB18,NaN,11.766373,0.026028,16495.0,469
cisbp__M01717,,differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down,NaN,TFAP4,11.125925,0.024766,16360.0,507
metacluster_177.3,,differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down,"MYF5, OLIG2, TCF21, OLIG3, TCF3, MYOD1, TFAP4, BHLHE22, MSC, ASCL2, ATOH1, PTF1A, FERD3L, MYOG, ZNF317, TCF12","NEUROD2, MYF5, TCF21, MYF6, MYOD1, TFAP4, ASCL2, PTF1A, MYOG, BHLHA15, ASCL5, TCF12",10.700348,0.023927,16549.0,424
neph__UW.Motif.0005,,differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down,NaN,NaN,10.549639,0.023630,16099.0,434
metacluster_193.2,,differential_peak_table_FDR05_annotated_matched_Belinostat | cytokine_down,"OLIG2, OLIG3, TAL1, TCF3, TFAP4, MSC, ZBTB18, ATOH1, ATOH7, NEUROG2, FERD3L, BHLHA15, TWIST1, TCF4, MSGN1, NEUROG1, NEUROD2","OLIG3, NEUROD4, NEUROD6, BHLHE23, TCF4, BHLHE22, TCF21, OLIG2, OLIG1, ATOH1, NEUROG2, TWIST1, NEUROD2, NEUROG3, ATOH7, NEUROG1, NEUROD1, TCF3, TCF12",10.395156,0.023325,16506.0,491


In [24]:
cistarget_results(cistarget_dict, name="differential_peak_table_FDR05_annotated_matched_Cytokine_up")

,Logo,Region_set,Direct_annot,Orthology_annot,NES,AUC,Rank_at_max,Motif_hits
jaspar__MA0249.2,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,"TCF21, TCF15, HAND2, HAND1, MSC, TCF23, TCF24, FERD3L, FIGLA, TWIST2, SCX, BHLHA9, TWIST1",5.549948,0.007117,6503.0,222
flyfactorsurvey__CG16778_SANGER_5_FBgn0003715,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,5.452620,0.007020,4906.0,197
yetfasco__YCR039C_1364,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,5.393151,0.006960,3374.0,142
hdpi__NR4A1,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NR4A1,NaN,5.284939,0.006853,16718.0,506
predrem__nrMotif1992,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,5.208355,0.006776,2443.0,121
predrem__nrMotif151,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,4.824330,0.006394,10185.0,310
flyfactorsurvey__esg_SANGER_2.5_FBgn0001981,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,4.812262,0.006382,16732.0,559
cisbp__M00127,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,4.795857,0.006365,7468.0,237
homer__YATGCATATRCATRT_OCT_OCT,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,4.591501,0.006162,1818.0,79
cisbp__M00642,,differential_peak_table_FDR05_annotated_matched_Cytokine_up,NaN,NaN,4.564606,0.006135,16297.0,472


In [25]:
cistarget_results(cistarget_dict, name="differential_peak_table_FDR05_annotated_matched_Cytokine_down")

,Logo,Region_set,Direct_annot,Motif_similarity_annot,Orthology_annot,Motif_similarity_and_Orthology_annot,NES,AUC,Rank_at_max


In [28]:

cistarget_results(cistarget_dict, name="differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up")

,Logo,Region_set,Direct_annot,Orthology_annot,NES,AUC,Rank_at_max,Motif_hits
hocomoco__MESP1_HUMAN.H11MO.0.D,,differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up,MESP1,NaN,5.150085,0.005529,16472.0,582
swissregulon__mm__Snai2,,differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up,NaN,SNAI2,5.022346,0.005426,16240.0,580
cisbp__M05844,,differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up,NaN,ZEB1,4.754101,0.005211,16735.0,646
flyfactorsurvey__esg_SANGER_2.5_FBgn0001981,,differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up,NaN,NaN,4.735574,0.005196,16528.0,612
swissregulon__mm__Snai1,,differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up,NaN,SNAI1,4.673344,0.005146,14452.0,542
hocomoco__SNAI1_HUMAN.H11MO.0.C,,differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up,SNAI1,NaN,4.559256,0.005054,16596.0,554
homer__VCAGGTRDRY_ZEB1,,differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up,ZEB1,NaN,4.558103,0.005053,15896.0,538
flyfactorsurvey__nau_da_SANGER_5_FBgn0000413,,differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up,NaN,"TCF4, TCF3, TCF12",4.223462,0.004785,16613.0,500
elemento__CACCTGG,,differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up,NaN,NaN,4.140312,0.004718,16721.0,526
flyfactorsurvey__sna_F2-4_SOLEXA_5,,differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up,NaN,NaN,4.061150,0.004655,15844.0,489


In [33]:
# Also running a permissive analysis (NES > 0.1 instead of 3) to be used for more quantitative comparisons between classes
cistarget_dict_perm = run_cistarget(ctx_db = cistarget_db,
                               region_sets = region_sets,
                               specie = 'homo_sapiens',
                               annotation_version = 'v10nr_clust',
                               path_to_motif_annotations = motif_anno,
                               auc_threshold = 0.005,
                               nes_threshold = 0.1,
                               rank_threshold = 0.05,
                               annotation = ['Direct_annot', 'Orthology_annot'],
                               n_cpu = 1,
                               _temp_dir='/lustre/scratch126/cellgen/team205/is10/tmp/')

2023-12-03 17:26:27,935 cisTarget    INFO     Reading cisTarget database
2023-12-03 17:27:54,715 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine_up which has 1356 regions
2023-12-03 17:28:54,828 cisTarget    INFO     Annotating motifs for differential_peak_table_FDR05_annotated_matched_Cytokine_up
2023-12-03 17:29:06,079 cisTarget    INFO     Getting cistromes for differential_peak_table_FDR05_annotated_matched_Cytokine_up
2023-12-03 17:29:18,968 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine_down which has 57254 regions
2023-12-03 17:45:05,613 cisTarget    INFO     Getting cistromes for differential_peak_table_FDR05_annotated_matched_Cytokine_down
2023-12-03 17:45:41,224 cisTarget    INFO     Running cisTarget for differential_peak_table_FDR05_annotated_matched_Cytokine + Belinostat_up which has 1682 regions
2023-12-03 17:47:32,044 cisTarget    INFO     Annotating motifs for diff

In [34]:
# Save
import pickle
with open((out_dir + 'cistarget_dict_NES0.1.pkl'), 'wb') as f:
  pickle.dump(cistarget_dict_perm, f)

In [35]:
## Also exporting results in a table format:
motif_enrichment_dict_perm = {key: cistarget_dict_perm[key].motif_enrichment for key in cistarget_dict_perm.keys()}
motif_enrichment_table_perm=pd.concat([motif_enrichment_dict_perm[key] for key in motif_enrichment_dict_perm.keys()], axis=0, sort=False)
motif_enrichment_table_perm.to_csv((out_dir + "cistarget_res_df_NES0.1.txt"), sep="\t")